In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from peft import PeftModel
import torch, os
from proccess import formatted_ds


/home/umugabekazi/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: Could not import module 'get_reporting_integration_callbacks'. Are this object's requirements defined correctly?

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from proccess import formatted_ds, tokenize_row

/home/umugabekazi/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: Could not import module 'Trainer'. Are this object's requirements defined correctly?

In [ ]:
# print(tokenized_ds.column_names)   # or for DatasetDict: print({k: v.column_names for k,v in tokenized_ds.items()})

In [ ]:
# tokenized_ds = tokenized_ds.remove_columns(["text", "input", "output", "instruction"])
# print(tokenized_ds[0])

In [ ]:
#load the model
model_name = "Featherless-Chat-Models/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Tokenize the formatted dataset using the loaded tokenizer and tokenize_row function
# This replaces the missing tokenized_ds

tokenized_ds = formatted_ds.map(lambda batch: tokenize_row(batch, tokenizer), batched=True)

# Optionally, remove unused columns if present
for col in ["text", "input", "output", "instruction"]:
    if col in tokenized_ds.column_names:
        tokenized_ds = tokenized_ds.remove_columns([col])

In [ ]:
#ensure use of gpu
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization for even less memory
    llm_int8_enable_fp32_cpu_offload=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",  # Load the model entirely on CPU to avoid OOM
    quantization_config=bnb_config,
)

model.resize_token_embeddings(len(tokenizer))
# Enable gradient checkpointing for further memory savings
model.gradient_checkpointing_enable()

# Configure LoRA
lora_config = LoraConfig(
    r=8,  # LoRA rank, adjust for memory
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # typical for Mistral/LLMs
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Attach LoRA adapters to the quantized model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading weights: 100%|██████████| 291/291 [05:04<00:00,  1.05s/it, Materializing param=model.norm.weight]                              


trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [ ]:
#training arguments
training_args = TrainingArguments(
    output_dir="./mistral_medical_model",
    per_device_train_batch_size=1,  
    gradient_accumulation_steps=4,  
    num_train_epochs=1,  
    learning_rate=0.00005,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=False,  # Disable fp16 since training is on CPU
    remove_unused_columns=False
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True
)

#trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,  # Use the tokenized dataset, not formatted_ds
    data_collator=data_collator
)


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# #train
# trainer.train()

Step,Training Loss
50,1.740355
100,0.723953
150,0.689705
200,0.656042
250,0.669318
300,0.660993
350,0.661161
400,0.641682
450,0.637613
500,0.613751


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from proccess import formatted_ds, tokenize_row

trainer.train(resume_from_checkpoint="./mistral_medical_model/checkpoint-2000")


ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434